## TUTORIAL 22 - Elastic contact problem
**_Keywords: geometrical parametrization_**

### 1. Introduction
This Tutorial addresses an application based on an idealized contact problem in linear elasticity with friction. We consider the equilibrium of a solid with a contact face that is partially sticking and partially sliding since we are interested in the initiation of dynamic sliding. In particular, we study the displacement of the idealized contact problem with friction under the applied load, satisfying the plane-strain linear elasticity equation in $\Omega_o$ as in the following picture:

<img src="https://github.com/RBniCS/RBniCS/raw/master/tutorials/22_elastic_contact/data/elastic_contact.png" width="70%"/>

We impose the following geometrically parametrized boundary conditions: homogeneous Dirichlet conditions on $\Gamma_{o,1} \cup \Gamma_{o,3}$ such that $u_2(\boldsymbol{\mu})=0$, homogeneous Dirichlet conditions on $\Gamma_{o,2}$ such that $u_1(\boldsymbol{\mu})=u_2(\boldsymbol{\mu})=0$, non-homogeneous Neumann conditions $\boldsymbol{\sigma}\boldsymbol{n}=-\boldsymbol{n}$ on $\Gamma_{o,5}$ and homogeneous Neumann conditions on any remaining boundaries.

Here $\boldsymbol{\sigma}$ represents the stress tensor on the domain $\Omega_o$ that is defined as follows
$$
\boldsymbol{\sigma}(\boldsymbol{u}_o;\boldsymbol{\mu})=\lambda_1(\boldsymbol{\mu})\;\text{tr}[\nabla^S\boldsymbol{u}_o]\;\boldsymbol{I}+2\;\lambda_2(\boldsymbol{\mu})\;\nabla^S\boldsymbol{u}_o,
$$
where $\nabla^S$ denotes the symmetric part of the gradient, while $\lambda_1$ and $\lambda_2$ are the Lamé constants for plane strain:
$$
\lambda_1=\frac{E \mu_1}{(1+\mu_1)(1-2\mu_1)},
 \quad \lambda_2=\frac{E}{2(1+\mu_1)},
$$
with $E=2$, since in all regions we assume an isotropic linear material.

This problem is characterized by $\boldsymbol{\mu}=(\mu_0,\mu_1)$ with $P=2$ components: $\mu_0$ controls the length of the two outer parts of the interface that slide in order to reach equilibrium, while $\mu_1$ characterizes the Poisson's ratio of the solid.

The parameter vector $\boldsymbol{\mu}$ is thus given by 
$$
\boldsymbol{\mu} = (\mu_0, \mu_1)
$$
on the parameter domain
$$
[0.05,0.45] \times [0.25,0.45].
$$

In order to obtain a faster (yet, provably accurate) approximation of the problem, and avoiding _any_ remeshing, we pursue a model reduction by means of a certified reduced basis reduced order method from a fixed reference domain.

In [ ]:
# Install FEniCS
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install-real.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

In [ ]:
# Install RBniCS
try:
    import rbnics
except ImportError:
    !pip3 install git+https://github.com/RBniCS/RBniCS.git
    import rbnics
import rbnics.utils.config
assert "dolfin" in rbnics.utils.config.config.get("backends", "required backends")

In [ ]:
# Download data files
!mkdir -p data
![ -f data/friction.xml ] || wget https://github.com/RBniCS/RBniCS/raw/master/tutorials/22_elastic_contact/data/friction.xml -O data/friction.xml
![ -f data/friction_facet_region.xml ] || wget https://github.com/RBniCS/RBniCS/raw/master/tutorials/22_elastic_contact/data/friction_facet_region.xml -O data/friction_facet_region.xml
![ -f data/friction_physical_region.xml ] || wget https://github.com/RBniCS/RBniCS/raw/master/tutorials/22_elastic_contact/data/friction_physical_region.xml -O data/friction_physical_region.xml
![ -f data/vertices_mapping.vmp ] || wget https://github.com/RBniCS/RBniCS/raw/master/tutorials/22_elastic_contact/data/vertices_mapping.vmp -O data/vertices_mapping.vmp

### 2. Parametrized formulation

Let $\boldsymbol{u}_o(\boldsymbol{\mu})$ be the displacement of the idealized contact problem with friction under the applied load, satisfying the plane-strain linear elasticity equation in $\Omega_o$.

We will directly provide a weak formulation for this problem: for a given parameter $\boldsymbol{\mu}\in\mathbb{P}$, find $\boldsymbol{u}_o(\boldsymbol{\mu})\in\mathbb{V}_o$ such that

$$a_o\left(\boldsymbol{u}_o(\boldsymbol{\mu}),\boldsymbol{v}_o;\boldsymbol{\mu}\right)=f_o(\boldsymbol{v}_o;\boldsymbol{\mu})\quad \forall \boldsymbol{v}_o\in\mathbb{V}_o$$

where

* the function space $\mathbb{V}_o$ is defined as
$$
\mathbb{V}_o=\{v_o \in H^1(\Omega): v\mid_{\Gamma_{o,1}\cup\Gamma_{o,2}\cup\Gamma_{o,3}} =0\}
$$

* the parametrized bilinear form $a_o(\cdot, \cdot; \boldsymbol{\mu}): \mathbb{V}_o \times \mathbb{V}_o \to \mathbb{R}$ is defined by
$$a(\boldsymbol{u}_o,\boldsymbol{v}_o;\boldsymbol{\mu})=\lambda_1(\boldsymbol{\mu}) \int_{\Omega_{o}} {\text{tr}[\nabla^S\boldsymbol{u}_o]\;\text{tr}[\nabla^S\boldsymbol{v}_o] \ d\boldsymbol{x} + \lambda_2(\boldsymbol{\mu}) \int_{\Omega_{o}} 2\;\nabla^S\boldsymbol{u}_o:\nabla^S\boldsymbol{v}_o} \ d\boldsymbol{x},$$
* the parametrized linear form $f_o(\cdot; \boldsymbol{\mu}): \mathbb{V}_o \to \mathbb{R}$ is defined by
$$f_o(\boldsymbol{v}_o;\boldsymbol{\mu})=-\int_{\Gamma_{o,5}} \boldsymbol{v}_o\cdot\boldsymbol{n} \ ds.$$

In [ ]:
from dolfin import *
from rbnics import *

## 3. Affine decomposition

In order to obtain an affine decomposition, we recast the problem on a fixed, parameter _independent_, reference domain $\Omega$. As reference domain which choose the one characterized by $\mu_0 = 0.2$ which we generate through the generate_mesh notebook provided in the _data_ folder.
Then, we pull back the problem to the reference domain $\Omega$.

In [ ]:
@PullBackFormsToReferenceDomain()
@AffineShapeParametrization("data/vertices_mapping.vmp")

class Friction(EllipticCoerciveProblem):

    #Default initialization of members
    def __init__(self,V,**kwargs):
        # Call the standard initialization
        EllipticCoerciveProblem.__init__(self, V, **kwargs)
        # ... and also store FEniCS data structures for assembly
        assert "subdomains" in kwargs
        assert "boundaries" in kwargs
        self.subdomains, self.boundaries = kwargs["subdomains"], kwargs["boundaries"]
        self.u = TrialFunction(V)
        self.v = TestFunction(V)
        self.dx = Measure("dx")(subdomain_data=subdomains)
        self.ds = Measure("ds")(subdomain_data=boundaries)
        # ...
        self.E = 2.0

    # Return custom problem name
    def name(self):
        return "Friction"

    # Return theta multiplicative terms of the affine expansion of the problem.
    def compute_theta(self, term):
        mu = self.mu
        E = self.E
        if term == "a":
            theta_a0 = E*mu[1] / ((1.0 + mu[1])*(1.0 - 2.0*mu[1])) #lambda lame coef
            theta_a1 = E / (2.0*(1.0 + mu[1])) #mu lame coef
            return (theta_a0, theta_a1)
        elif term == "f":
            theta_f0 = 1.0
            return (theta_f0,)
        elif term == "s":
            theta_s0 = 1.0
            return (theta_s0,)
        else:
            raise ValueError("Invalid term for compute_theta().")

    # Return forms resulting from the discretization of the affine expansion of the problem operators.
    def assemble_operator(self, term):
        v = self.v
        dx = self.dx
        if term == "a":
            u = self.u
            a0 = tr(sym(nabla_grad(u)))*tr(sym(nabla_grad(v)))*dx
            a1 = 2.0*inner(sym(nabla_grad(u)), sym(nabla_grad(v)))*dx
            return (a0, a1)
        elif term == "f":
            ds = self.ds
            u = self.u
            f0 = inner(Constant((0.0, -1.0)), v)*ds(5)
            return (f0,)
        elif term == "s":
            ds = self.ds
            u = self.u
            s0 = inner(Constant((0.0, -1.0)), v)*ds(5)
            return (s0,)
        elif term == "dirichlet_bc":
            bc0 = [DirichletBC(self.V.sub(1), Constant(0.0), self.boundaries, 1),
                  DirichletBC(self.V.sub(1), Constant(0.0), self.boundaries, 3),
                  DirichletBC(self.V, Constant((0.0, 0.0)), self.boundaries, 2)]
            return (bc0,)
        elif term == "inner_product":
            u = self.u
            x0 = inner(u, v)*dx + inner(grad(u), grad(v))*dx
            return (x0,)
        else:
            raise ValueError("Invalid term for assemble_operator().")

## 4. Main program
### 4.1. Read the mesh for this problem
The mesh was generated by the [data/generate_mesh.ipynb](data/generate_mesh.ipynb) notebook.

In [ ]:
mesh = Mesh("data/friction.xml")
subdomains = MeshFunction("size_t", mesh, "data/friction_physical_region.xml")
boundaries = MeshFunction("size_t", mesh, "data/friction_facet_region.xml")

### 4.2. Create Finite Element space (Lagrange P2)

In [ ]:
V = VectorFunctionSpace(mesh, "Lagrange", 2)

### 4.3. Allocate an object of the Subfin class

In [ ]:
problem = Friction(V, subdomains=subdomains, boundaries=boundaries)
mu_range = [(0.05, 0.45),(0.25, 0.45)]
problem.set_mu_range(mu_range)

### 4.4. Prepare reduction with a POD-Galerkin method

In [ ]:
reduction_method = PODGalerkin(problem)
reduction_method.set_Nmax(50)
reduction_method.set_tolerance(1e-4)

### 4.5. Perform the offline phase

In [ ]:
reduction_method.initialize_training_set(100)
reduced_problem = reduction_method.offline()

### 4.6. Perform an online solve

In [ ]:
online_mu = (0.37, 0.26)
reduced_problem.set_mu(online_mu)
reduced_solution = reduced_problem.solve()
plot(reduced_solution, reduced_problem=reduced_problem)

### 4.7. Perform an error analysis

In [ ]:
reduction_method.initialize_testing_set(100)
reduction_method.error_analysis(filename="error_analysis")

### 4.8. Perform a speedup analysis

In [ ]:
reduction_method.speedup_analysis(filename="speedup_analysis")

### 5. Assignments


1.	Consider a different configuration of the domain, in which the external force pushes on the left boundary $\Gamma_{o,6}$, instead of being loaded on the upper boundary $\Gamma_{o,5}$. Edit the mesh generation and change the problem to account for the modified boundary conditions. Discuss the performance of the reduction in terms of errors and speedups.

2. Change the model reduction technique from POD-Galerkin to certified reduced basis. Compare the results of the error analysis and speedup analysis for the two reduction techniques.